<a href="https://colab.research.google.com/github/nezamtrm/pneumonia_detection/blob/main/load_split_xray_folders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
%cd /content/drive/MyDrive/pneumonia_detection

[Errno 2] No such file or directory: '/content/drive/MyDrive/pneumonia_detection'
/content


In [ ]:
od.download('https://www.kaggle.com/datasets/artyomkolas/3-kinds-of-pneumonia')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nedataghinezhad
Your Kaggle Key: ··········


100%|██████████| 3.49G/3.49G [00:56<00:00, 65.9MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
colab_dataset_path = '/content/3-kinds-of-pneumonia/'

drive_dataset_path = '/content/gdrive/My Drive/Colab Notebooks/dataset'

In [ ]:
import shutil

shutil.copytree(colab_dataset_path, drive_dataset_path)

In [ ]:
data_dir = '/content/gdrive/MyDrive/Colab Notebooks/dataset/Curated X-Ray Dataset'
class_names = ['COVID-19', 'Normal', 'Pneumonia-Bacterial', 'Pneumonia-Viral']

In [ ]:
all_image_fps = []
all_labels = []

In [ ]:
for class_index, class_name in enumerate(class_names):
    class_dir = os.path.join(data_dir, class_name)
    image_fps = [os.path.join(class_dir, file) for file in os.listdir(class_dir)]
    all_image_fps.extend(image_fps)
    all_labels.extend([class_index] * len(image_fps))

In [ ]:
train_size = 0.7  # Percentage of data for training
valid_size = 0.15  # Percentage of data for validation
test_size = 0.15  # Percentage of data for testing

# Split data into training and temp sets
train_image_fps, temp_image_fps, train_labels, temp_labels = train_test_split(
    all_image_fps, all_labels, train_size=train_size, stratify=all_labels, random_state=42
)

# Split temp set into validation and test sets
valid_image_fps, test_image_fps, valid_labels, test_labels = train_test_split(
    temp_image_fps, temp_labels, test_size=test_size / (test_size + valid_size),
    stratify=temp_labels, random_state=42
)


In [ ]:
test_data_dir0 = os.path.join(data_dir, 'test_set')
if not os.path.exists(test_data_dir0):
    os.makedirs(test_data_dir0)

valid_data_dir0 = os.path.join(data_dir, 'valid_set')
if not os.path.exists(valid_data_dir0):
    os.makedirs(valid_data_dir0)

train_data_dir0 = os.path.join(data_dir, 'train_set')
if not os.path.exists(train_data_dir0):
    os.makedirs(train_data_dir0)


In [ ]:
for image_fp in valid_image_fps:
    class_name = class_names[valid_labels[valid_image_fps.index(image_fp)]]
    vaild_data_dir = os.path.join(valid_data_dir0, class_name)
    if not os.path.exists(vaild_data_dir):
        os.makedirs(vaild_data_dir)
    shutil.copy(image_fp, vaild_data_dir)


In [ ]:
for image_fp in test_image_fps:
    class_name = class_names[test_labels[test_image_fps.index(image_fp)]]
    test_data_dir = os.path.join(test_data_dir0, class_name)
    if not os.path.exists(test_data_dir):
        os.makedirs(test_data_dir)
    shutil.copy(image_fp, test_data_dir)


In [ ]:
for image_fp in train_image_fps:
    class_name = class_names[train_labels[train_image_fps.index(image_fp)]]
    train_data_dir = os.path.join(train_data_dir0, class_name)
    if not os.path.exists(train_data_dir):
        os.makedirs(train_data_dir)
    shutil.copy(image_fp, train_data_dir)

In [ ]:
test_data_dir0 = '/content/gdrive/MyDrive/Colab Notebooks/dataset/Curated X-Ray Dataset/test_set'
train_data_dir0 = '/content/gdrive/MyDrive/Colab Notebooks/dataset/Curated X-Ray Dataset/train_set'
valid_data_dir0 = '/content/gdrive/MyDrive/Colab Notebooks/dataset/Curated X-Ray Dataset/valid_set'

In [ ]:
datagen = ImageDataGenerator(rescale=1.0 / 255)


In [ ]:
# Set batch size and image size
batch_size = 32
image_size = (224, 224)

train_dataset = datagen.flow_from_directory(
    train_data_dir0,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_names,
    shuffle=True,
    subset='training'  # Use this if you have separate train and validation folders
)

validation_dataset = datagen.flow_from_directory(
    valid_data_dir0,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_names,
    shuffle=False,
)


test_dataset = datagen.flow_from_directory(
    test_data_dir0,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_names,
    shuffle=False  # Keep the original order of test images
)
